<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"></ul></div>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import sys

from datetime import datetime, timedelta
from tqdm import tqdm

LEFT_PATH = '../data/bb_left_forearm_csv/'
RIGHT_PATH = '../data/bb_right_forearm_csv/'
TABLE_PATH = '../data/taiku_tables/'
PATH = '../data/bb_capture/'
OUTPUT_PATH = '../output/'
PROCESSED_PATH = '../data/'

In [2]:
df = pd.read_csv(TABLE_PATH + 'taiko_play.csv')
tmp_df = pd.read_csv(TABLE_PATH + 'taiko_song.csv', dtype={
    'song_length': np.int16
})
df = df.merge(tmp_df, how='left', left_on='song_id', right_on='song_id')
tmp_df = pd.read_csv(TABLE_PATH + 'taiko_drumer.csv')
df = df.merge(tmp_df, how='left', left_on='drumer_id', right_on='id')
local_time = df['start_time'][0]
df

,pid,drumer_id,song_id,performance_order,start_time,song_name,song_length,id,name,handedness,gender,armlength
0,1,1,1,1,3/27/2018 11:53:08,夢をかなえてドラえもん,90,1,陳啟元,0,M,26.0
1,2,1,1,2,3/27/2018 11:55:29,夢をかなえてドラえもん,90,1,陳啟元,0,M,26.0
2,3,1,1,3,3/27/2018 11:57:53,夢をかなえてドラえもん,90,1,陳啟元,0,M,26.0
3,4,1,2,1,3/27/2018 12:27:53,ウィーアー！,120,1,陳啟元,0,M,26.0
4,5,1,2,2,3/27/2018 12:30:18,ウィーアー！,120,1,陳啟元,0,M,26.0
5,6,1,2,3,3/27/2018 12:32:45,ウィーアー！,120,1,陳啟元,0,M,26.0
6,7,1,3,1,3/27/2018 12:35:45,六兆年と一夜物語\n,150,1,陳啟元,0,M,26.0
7,8,1,3,2,3/27/2018 12:38:33,六兆年と一夜物語\n,150,1,陳啟元,0,M,26.0
8,9,1,3,3,3/27/2018 12:41:25,六兆年と一夜物語\n,150,1,陳啟元,0,M,26.0
9,10,1,4,1,3/27/2018 12:44:52,残酷な天使のテーゼ,90,1,陳啟元,0,M,26.0


In [3]:
def get_hwclock_time(local_time, delta=0):
    d = datetime.strptime(local_time, "%m/%d/%Y %H:%M:%S")
    d = d + timedelta(seconds=int(delta))
    return time.mktime(d.timetuple())

In [4]:
files = next(os.walk(LEFT_PATH))[2]
left_df = [
    pd.read_csv(LEFT_PATH + filename, dtype={
        'timestamp': np.float64
    }) for filename in files
]
left_df = pd.concat(left_df, ignore_index=True)
left_df.drop('key', axis=1, inplace=True)
files

['data_2018_03_27_13_25_43.csv',
 'data_2018_03_28_15_52_48.csv',
 'data_2018_03_27_14_48_28.csv',
 'data_2018_03_28_16_59_47.csv',
 'data_2018_03_27_11_37_57.csv',
 'data_2018_03_28_14_52_45.csv',
 'data_2018_03_27_16_12_02.csv',
 'data_2018_03_31_15_29_36.csv']

In [5]:
files = next(os.walk(RIGHT_PATH))[2]
right_df = [
    pd.read_csv(RIGHT_PATH + filename, dtype={
        'timestamp': np.float64
    }) for filename in files
]
right_df = pd.concat(right_df, ignore_index=True)
right_df.drop('key', axis=1, inplace=True)
files

['data_2018_03_27_11_40_03.csv',
 'data_2018_03_28_16_59_37.csv',
 'data_2018_03_27_14_50_24.csv',
 'data_2018_03_28_14_52_43.csv',
 'data_2018_03_31_15_29_27.csv',
 'data_2018_03_27_16_14_52.csv',
 'data_2018_03_28_15_52_55.csv',
 'data_2018_03_27_13_20_28.csv']

In [6]:
for col in left_df:
    if col != 'timestamp' and col != 'wall_time':
        print('retrieving column %s ...' % (str(col)))
        sys.stdout.flush()
        
        # access particular folder, if doesn't exist, create it
        directory = OUTPUT_PATH + str(col)   
        if not os.path.exists(directory):
            os.makedirs(directory)
            
        for i in tqdm(range(len(df))):
            plt.figure(figsize=(15, 8))
            row = df.iloc[i]
            start_time = get_hwclock_time(row['start_time'])
            end_time = get_hwclock_time(row['start_time'], row['song_length'])
            play_df = left_df[(left_df['timestamp'] >= start_time) & (left_df['timestamp'] <= end_time)]
            plt.plot(play_df['timestamp'], play_df[col], label='left')
            
            play_df = right_df[(right_df['timestamp'] >= start_time) & (right_df['timestamp'] <= end_time)]
            plt.plot(play_df['timestamp'], play_df[col], label='right')
            
            plt.legend()
            save_name = '%s who_id:%d song_id:%d order:%d' % (col, row['id'], row['song_id'], row['performance_order'])
            plt.title(save_name)
            plt.savefig(directory + '/' + save_name + '.png')
            plt.close()

retrieving column imu_temp ...


100%|██████████| 104/104 [00:12<00:00,  8.46it/s]

retrieving column imu_ax ...



100%|██████████| 104/104 [00:13<00:00,  7.79it/s]

retrieving column imu_ay ...


...


100%|██████████| 104/104 [00:12<00:00,  8.40it/s]

retrieving column baro_temp ...



100%|██████████| 104/104 [00:10<00:00,  9.46it/s]

retrieving column baro ...



100%|██████████| 104/104 [00:13<00:00,  7.96it/s]
